In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import corner
from emcee import EnsembleSampler

# Constants and measurement uncertainties
const_s = 1370  # Solar radiation constant (W/m^2)
sigma_sb = 5.67e-8  # Stefan-Boltzmann constant (W/m^2/K^4)
sigma_T = 0.5  # Uncertainty in observed temperature (K)
sigma_y_co2 = 0.05  # Uncertainty in y_CO2

# Observed data
observed_T = df['Temperature (K)'].values
observed_y_co2 = df['y_CO2'].values

# Prior means and uncertainties for parameters
alpha_0 = 0.2  # Initial guess for alpha
f0_0 = 0.6  # Initial guess for f0
f1_0 = 0.002  # Initial guess for f1
sigma_alpha = 0.1  # Uncertainty in alpha
sigma_f0 = 0.1  # Uncertainty in f0
sigma_f1 = 0.1  # Uncertainty in f1

# Step 1: Define the prior function
def prior(alpha, f0, f1):
    # Uniform priors for alpha and f0
    if not (0 <= alpha <= 1) or not (0 <= f0 <= 1):
        return -np.inf  # Log probability is -inf for out-of-bounds values
    
    # Gaussian prior for f1
    prior_f1 = norm.logpdf(f1, loc=f1_0, scale=sigma_f1)

    # Log-space uniform priors for alpha and f0 (constant within bounds)
    prior_alpha = 0  # Uniform prior in [0, 1] has no contribution in log-space
    prior_f0 = 0     # Same for f0 in [0, 1]
    
    return prior_alpha + prior_f0 + prior_f1

# Step 2: Define the likelihood function
def likelihood(alpha, f0, f1, observed_T, observed_y_co2):
    # Calculate theoretical T using the given equation
    denominator = sigma_sb * (1 - (f0 + f1 * observed_y_co2) / 2)
    if np.any(denominator <= 0):  # Check for valid denominator
        return -np.inf

    theoretical_T = ((const_s * (1 - alpha) / 4) / denominator) ** (1 / 4)
    return np.sum(norm.logpdf(observed_T, loc=theoretical_T, scale=sigma_T))

# Step 3: Define the posterior function
def posterior(params, observed_T, observed_y_co2):
    alpha, f0, f1 = params
    prior_val = prior(alpha, f0, f1)
    if prior_val == -np.inf:
        return -np.inf
    likelihood_val = likelihood(alpha, f0, f1, observed_T, observed_y_co2)
    return prior_val + likelihood_val

# Step 4: Initialize MCMC sampler
nwalkers = 50
nsteps = 1000

# Uniform sampling for alpha and f0 within [0, 1]
alpha_init = np.random.uniform(0, 1, size=nwalkers)
f0_init = np.random.uniform(0, 1, size=nwalkers)

# Gaussian sampling for f1 based on its prior
f1_init = np.random.normal(f1_0, sigma_f1, size=nwalkers)

# Combine into initial positions array
initial_positions = np.vstack([alpha_init, f0_init, f1_init]).T

# Initialize sampler
sampler = EnsembleSampler(
    nwalkers,
    3,  # Sampling three parameters: alpha, f0, and f1
    posterior,
    args=(observed_T, observed_y_co2)
)

# Step 5: Run MCMC sampling
sampler.run_mcmc(initial_positions, nsteps, progress=True)
samples = sampler.get_chain(discard=100, thin=10, flat=True)

# Analyze MCMC results
alpha_mean, f0_mean, f1_mean = np.mean(samples, axis=0)
alpha_std, f0_std, f1_std = np.std(samples, axis=0)

print(f"Estimated alpha: Mean = {alpha_mean:.4f}, Std = {alpha_std:.4f}")
print(f"Estimated f0: Mean = {f0_mean:.4f}, Std = {f0_std:.4f}")
print(f"Estimated f1: Mean = {f1_mean:.6f}, Std = {f1_std:.6f}")

# Corner plot
fig_full = corner.corner(samples, labels=["alpha", "f0", "f1"], truths=[alpha_mean, f0_mean, f1_mean])
fig_full.savefig("full_corner_plot.png", dpi=300)
plt.show()
